# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Establishing connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Creating queries to return answers to the following three questions of the data

### 1. Query: Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## First, creating a new table for Query1. Using sessionId as partition key and itemInSession as clustering column as queries are requiring both fields for quick retrieval
query = "CREATE TABLE IF NOT EXISTS song_sessions"
query = query + "(artist text,itemInSession int,length float,sessionId int,song text,PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        ## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_sessions (artist,itemInSession,length,sessionId,song)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query,(line[0],int(line[3]),float(line[5]),int(line[8]),line[9]))

In [10]:
## SELECT statement to return the data for Query 1
query = "select artist,song,length FROM song_sessions WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Query: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [21]:
## First, creating a new table for Query2. Using userId and sessionId as composite partition keys to make queries on both fields more performant. 
## Eventually adding the itemInSession field as clustering column to get records sorted by this field descendingly as default

query = "CREATE TABLE IF NOT EXISTS user_sessions"
query = query + "(artist text,itemInSession int,sessionId int,userId int,song text, firstName text,lastName text, PRIMARY KEY ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
print("++++Table user_sessions created++++")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (artist,itemInSession,sessionId,userId,song,firstName,lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query,(line[0],int(line[3]),int(line[8]),int(line[10]),line[9],line[1],line[4]))

print("++++Data inserted into user_sessions table++++")
                    

Error from server: code=2200 [Invalid query] message="unconfigured table user_sessions"
++++Table user_sessions created++++
++++Data inserted into user_sessions table++++


In [22]:
## SELECT statement to return the data for Query 2
query = "select artist,song,firstName,lastName from user_sessions WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Query: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [26]:
## First, creating a new table for Query3. Using song and userId composite partition keys as multiple users may listen to the same song

query = "CREATE TABLE IF NOT EXISTS song_plays"
query = query + "(song text, firstName text,lastName text,userId int, PRIMARY KEY (song,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
print("++++Table song_plays created++++")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        ## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_plays (song,firstName,lastName,userId)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query,(line[9],line[1],line[4],int(line[10])))

print("++++Data inserted into song_plays table++++")

                    

++++Table song_plays created++++
++++Data inserted into song_plays table++++


In [27]:
## SELECT statement to return the data for Query 3
query = "select firstName,lastName from song_plays WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [25]:
query = "DROP TABLE song_sessions"
try:
    session.execute(query)
    print("song_sessions table dropped")
except Exception as e:
    print(e)

query = "DROP TABLE user_sessions"
try:
    session.execute(query)
    print("user_sessions table dropped")
except Exception as e:
    print(e)

query = "DROP TABLE song_plays"
try:
    session.execute(query)
    print("song_plays table dropped")
except Exception as e:
    print(e)


song_sessions table dropped
user_sessions table dropped
song_plays table dropped


### Close the session and cluster connection¶

In [57]:
session.shutdown()
cluster.shutdown()